

# Neptune MultiModel (Ask KG Data Product Questions)
This notebook shows the movie example from my talk/blog post on using Amazon Neptune to help model a multi-model database solution.

Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved. SPDX-License-Identifier: MIT-0

Begin by setting up. Run the next cell to instruct the notebook to get Neptune data from S3 bucket provisioned for you.

In [ ]:
import os
import subprocess

stream = os.popen("source ~/.bashrc ; echo $STAGE_BUCKET; echo $M2C_ANALYSIS_BUCKET")
lines=stream.read().split("\n")
STAGING_BUCKET=lines[0]
STAGING_BUCKET

## Obtain UML files
TODO describe


In [ ]:
%%bash -s "$STAGING_BUCKET"

mkdir -p uml
mkdir -p mmgen
cd uml
aws s3 cp s3://$1/uml . --recursive


## Extract data products/impl from UML files
UML is represented in the files in XML Metadata Interchange (XMI) form. Let's extract the main details from those files into Python maps. In the cell that follows we will build RDF from what we extraced


In [ ]:
import sys
import json
import xml.etree.ElementTree as ET

# Namespace stuff for XMI parsing
NS={
    "uml": "http://www.omg.org/spec/UML/20131001",
    "xmi": "http://www.omg.org/spec/XMI/20131001",
    "MMProfile": "http://www.magicdraw.com/schemas/MMProfile.xmi"
}
XMI_ID = "{" + NS['xmi'] + "}id"
XMI_IDREF = "{" + NS['xmi'] + "}idref"

def get_attrib(elem, name):
    if name in elem.attrib:
        return elem.attrib[name]
    else:
        return ""
        
def add_tagval(stereotype, tag, val):
    if tag in stereotype['tags']:
        stereotype['tags'][tag].append(val)
    else:
        stereotype['tags'][tag] = [val]

def get_tags(elem, tags, stereotype):
    for tag in tags:
        aval = get_attrib(elem, tag)
        if len(aval) > 0:
            add_tagval(stereotype, tag, aval)
        tagvs = elem.findall(tag)
        for tagv in tagvs:
            add_tagval(stereotype, tag, tagv.text)

umlextract = {}

def extract(filename):
    path = f"uml/{filename}"
    packages = {}
    classes = {}
    usages = {}
    props = {}
    imports = {}
    datatypes = {}
    enums = {}
    
    print("Parsing " + path)
    tree = ET.parse(path)

    # packages
    for elem in tree.findall("uml:Model//packagedElement[@xmi:type='uml:Package']", NS):
        id = get_attrib(elem, XMI_ID)
        name = get_attrib(elem, 'name')
        parent = tree.findall(f'.//packagedElement[@xmi:id="{id}"]...', NS)
        parent_id = get_attrib(parent[0], XMI_ID) if len(parent) == 1 else  "";
        packages[id] = { 'id': id, 'name': name, 'parent': parent_id, 'stereotypes': []}

    # package imports
    for elem in tree.findall("uml:Model//packageImport[@xmi:type='uml:PackageImport']", NS):
        id = get_attrib(elem, XMI_ID)
        name = get_attrib(elem, 'name')
        parent = tree.findall(f'.//packagedElement[@xmi:id="{id}"]...', NS)
        parent_id = get_attrib(parent[0], XMI_ID) if len(parent) == 1 else  "";
        ip = elem.find("importedPackage", NS)
        href = get_attrib(ip, "href")
        imports[id] = { 'id': id, 'name': name, 'parent': parent_id, 'href': href, 'stereotypes': []}

    # classes
    for elem in tree.findall("uml:Model//packagedElement[@xmi:type='uml:Class']", NS):
        id = get_attrib(elem, XMI_ID)
        name = get_attrib(elem, 'name')
        parent = tree.findall(f'.//packagedElement[@xmi:id="{id}"]...', NS)
        parent_id = get_attrib(parent[0], XMI_ID) if len(parent) == 1 else  "";

        attribs = elem.findall("ownedAttribute", NS)
        class_attribs = {}
        for a in attribs:
            aid = get_attrib(a, XMI_ID)
            aname = get_attrib(a, 'name')
            aggregation = get_attrib(a, 'aggregation')
            atype = get_attrib(a, 'type')
            assoc = get_attrib(a, 'association')
            props[aid] = id #map property to clazz
            class_attribs[aid] = { 'id': aid, 'name': aname, 'aggregation': aggregation, 'type': atype, 'association': assoc, 'stereotypes': []}

        classes[id] = { 'id': id, 'name': name, 'parent': parent_id,  'usages': {}, 'stereotypes': [], 'properties': class_attribs}

    # datatypes
    for elem in tree.findall("uml:Model//packagedElement[@xmi:type='uml:DataType']", NS):
        id = get_attrib(elem, XMI_ID)
        name = get_attrib(elem, 'name')
        datatypes[id] = { 'id': id, 'name': name}
        
    # enums
    for elem in tree.findall("uml:Model//packagedElement[@xmi:type='uml:Enumeration']", NS):
        id = get_attrib(elem, XMI_ID)
        name = get_attrib(elem, 'name')
        lits = []
        for lit_node in elem.findall("./ownedLiteral", NS ):
            lits.append(get_attrib(lit_node, 'name'))
            
        enums[id] = { 'id': id, 'name': name, 'literals': lits}


    # usages
    for elem in tree.findall("uml:Model//packagedElement[@xmi:type='uml:Usage']", NS):
        id = get_attrib(elem, XMI_ID)
        target = get_attrib(elem.find('supplier'), XMI_IDREF)
        source = get_attrib(elem.find('client'), XMI_IDREF)
        targetHref = get_attrib(elem.find('supplier'),'href')
        usages[id] = source
        if not(source in classes):
            print(f"Warn: usage broken ref {source}")
        else:
            u = {'id': id, 'target': target, 'targetHref': targetHref, 'stereotypes': []}
            classes[source]['usages'][id] = u

    # data products - and link to classes
    for elem in tree.findall("./MMProfile:DataProduct", NS):
        id = get_attrib(elem, XMI_ID)
        clazz_id = get_attrib(elem, 'base_Class')
        if not(clazz_id in classes):
            print(f"Warn: stereotype (id) broken ref {clazz_id}")
        else:
            classes[clazz_id]['isProduct'] = True

    # impls, and link to classes
    for elem in tree.findall("./MMProfile:DataProductImpl", NS):
        id = get_attrib(elem, XMI_ID)
        clazz_id = get_attrib(elem, 'base_Class')
        if not(clazz_id in classes):
            print(f"Warn: stereotype (id) broken ref {clazz_id}")
        else:
            classes[clazz_id]['isImpl'] = True

    # usage rels
    urels = {'joins': ['joinAttrib', 'myAttrib'], 
        'refersTo': ['refersAttrib', 'myAttrib'], 
        'hasImpl': [], 'caches': [], 'copies': [], 'locatedIn':[],
        'similarTo': ['simReason', 'simAlgorithm'], 
        'config': ['configKV'],
        'hasSource': ['integrationType', 'sourceDesc', 'sourceDataSet', 'sourceEventType'],
        'federates': ['fedURI']
        }
    for u in urels:
        for elem in tree.findall(f"./MMProfile:{u}", NS):
            id = get_attrib(elem, XMI_ID)
            usage = get_attrib(elem, 'base_Usage')
            elem_id = get_attrib(elem, 'base_Element')
            stereotype = {'name': u, 'tags': {}}
            get_tags(elem, urels[u], stereotype)

            if usage in usages:
                source = usages[usage]
                classes[source]['usages'][usage]['stereotypes'].append(stereotype)
            elif elem_id in usages:
                source = usages[elem_id]
                classes[source]['usages'][elem_id]['stereotypes'].append(stereotype)

    # stereotypes at class/package level
    srels = {'awsService': ['service'], 
        'awsResource': ['resource'], 
        'usagePattern': ['pattern'], 
        'config': ['configKV'],
        'hasSource': ['integrationType', 'sourceDesc', 'sourceDataSet', 'sourceEventType'],
        'federates': ['fedURI']
        }
    for u in srels:
        for elem in tree.findall(f"./MMProfile:{u}", NS):
            id = get_attrib(elem, XMI_ID)
            clazz = get_attrib(elem, 'base_Class')
            pkg = get_attrib(elem, 'base_Package')
            pkgi = get_attrib(elem, 'base_PackageImport')
            elem_id = get_attrib(elem, 'base_Element')
            stereotype = {'name': u, 'tags': {}}
            get_tags(elem, srels[u], stereotype)
            if clazz in classes:
                classes[clazz]['stereotypes'].append(stereotype)
            elif pkg in packages:
                packages[pkg]['stereotypes'].append(stereotype)
            elif pkgi in imports:
                imports[pkgi]['stereotypes'].append(stereotype)
            elif elem_id in classes:
                classes[elem_id]['stereotypes'].append(stereotype)
            elif elem_id in packages:
                packages[elem_id]['stereotypes'].append(stereotype)

    # property-level
    prels = {'productKey': [], 'config': ['configKV'] }
    for u in prels:
        for elem in tree.findall(f"./MMProfile:{u}", NS):
            id = get_attrib(elem, XMI_ID)
            property = get_attrib(elem, 'base_Property')
            elem_id = get_attrib(elem, 'base_Element')
            stereotype = {'name': u, 'tags': {}}
            get_tags(elem, prels[u], stereotype)
            if property in props:
                clazz = props[property]
                classes[clazz]['properties'][property]['stereotypes'].append(stereotype)
            elif elem_id in props:
                clazz = props[elem_id]
                classes[clazz]['properties'][property]['stereotypes'].append(stereotype)
                
    print("done")
    umlextract[filename] = {'packages': packages, 'classes': classes, 
        'usages': usages, 'props': props, 'imports': imports, 'datatypes': datatypes, 'enums': enums}


UML_FILES = ['DataLake.xml', 'VideoAnalysis.xml', 'StoryAnalysis.xml', 'MovieDoc.xml', 'KnowledgeGraph.xml']
for uf in UML_FILES:
    extract(uf)

umlextract

## Combine UML output
We collected lots of different things from UML. Let's bring them together into a list of products and impls.

In [ ]:
file_prod_impl_map = None
file_prod_impl_map = {}


# Build properties for the given class at the given level
def build_properties(filename, clazz, level):
    props = umlextract[filename]['classes'][clazz]['properties']
    for p in props:
        name = props[p]['name']
        type_id = props[p]['type']
        stereotypes = props[p]['stereotypes']
        type_name = ""  
        subtype = None
        literals = None
        
        if type_id in umlextract[filename]['classes'] and \
            not('isProduct' in umlextract[filename]['classes'][type_id]) and \
            not('isImpl' in umlextract[filename]['classes'][type_id]):
            
            type_name = umlextract[filename]['classes'][type_id]['name']
            subtype = build_properties(filename, type_id, [])
            
        if type_id in umlextract[filename]['datatypes']:
            type_name = umlextract[filename]['datatypes'][type_id]['name']
            
        if type_id in umlextract[filename]['enums']:
            type_name = umlextract[filename]['enums'][type_id]['name']
            literals = umlextract[filename]['enums'][type_id]['literals']
            
        prop_entry = {
            'name': name,
            'type': type_name,
            'subtype': subtype,
            'literals': literals,
            'stereotypes': stereotypes
        }
        level.append(prop_entry)
        
    return level
    
def find_targets(filename, target_id, target_href):
    target_spec = {'target': None, 'targets': [], 'target_file': None}
    if len(target_id) > 0:
        if target_id in umlextract[filename]['classes']:
            target_spec['target'] = umlextract[filename]['classes'][target_id]['name']
            target_spec['targets'].append(target_spec['target'])
            target_spec['target_file'] = filename
        else:
            print(f"Warn: broken target in usage {u}")
    elif len(target_href) > 0:                
        toks = target_href.split("#")
        if len(toks) == 2:
            target_spec['target_file'] = toks[0]
            target_elem = toks[1]
            if target_spec['target_file'] in umlextract:
                if target_elem in umlextract[target_spec['target_file']]['classes']:
                    target_spec['targets'].append(umlextract[target_spec['target_file']]['classes'][target_elem]['name'])
                elif target_elem in umlextract[target_spec['target_file']]['packages']:
                    for c in file_prod_impl_map[target_spec['target_file']]['products']:
                        if target_elem in c['packageAncestry']:
                            target_spec['targets'].append(c['name'])

            else:
                print(f"Warn: unknown target file in usage {u}")
        else:
            print(f"Warn: unexpected target ref in usage {u}")
    if len(target_spec['targets']) == 1:
        target_spec['target'] = target_spec['targets'][0]
    return target_spec

# Bring together all properties, usages, and stereotypes of products an impls for the given UML filename
def combine(filename):
    print("Combining " + filename)
    products = []
    impls = []
    
    # consider all the classes in the UML file
    for c in umlextract[filename]['classes']:
        uobj = umlextract[filename]['classes'][c]
        obj = {
            'name': uobj['name'],
            'stereotypes': [],
            'properties': build_properties(filename, c, []),
            'packageAncestry': []
        }

        # remember if it's a product or impl
        if 'isProduct' in uobj:
            products.append(obj)
        if 'isImpl' in uobj:
            impls.append(obj)
            
        # class stereotypes - incorporate
        for st in uobj['stereotypes']:
            obj['stereotypes'].append({
                'name': st['name'],
                'tags': st['tags']
            })

        # stereotype package imports
        for imp in umlextract[filename]['imports']:   
            target_href = umlextract[filename]['imports'][imp]['href']
            target_spec = find_targets(filename, "", target_href)

            for s in umlextract[filename]['imports'][imp]['stereotypes']:    
                print("Add import " + s['name'])
                obj['stereotypes'].append({
                    'name': s['name'],
                    'tags': s['tags'],
                    'targetClass': target_spec['target'],
                    'targetClasses': target_spec['targets'],
                    'targetFile': target_spec['target_file']
                })

        for u in uobj['usages']:
            target_id = uobj['usages'][u]['target']
            target_href = uobj['usages'][u]['targetHref']
            target_spec = find_targets(filename, target_id, target_href)
            
            for s in uobj['usages'][u]['stereotypes']:
                print("Add usage  " + s['name'])
                obj['stereotypes'].append({
                    'name': s['name'],
                    'tags': s['tags'],
                    'targetClass': target_spec['target'],
                    'targetClasses': target_spec['targets'],
                    'targetFile': target_spec['target_file']
                })
            
        # inherit stereotypes from ancestor packages
        currobj=uobj
        while len(currobj['parent']) > 0 and currobj['parent'] in umlextract[filename]['packages']:       
            currobj = umlextract[filename]['packages'][uobj['parent']]
            obj['packageAncestry'].append(currobj['id'])
            for s in currobj['stereotypes']:
                print("Add ancestry " + s['name'])
                obj['stereotypes'].append(s)
                
    file_prod_impl_map[filename] = {'products': products, 'impls': impls}

for uf in UML_FILES:
    combine(uf)

file_prod_impl_map

## Look at moviedoc products
We'll look at MovieDocument. 
Notice: stereotypes (source, join, impl), properties (a few levels deep in some cases)

When we convert to RDF, we won't take all properties. 
We'll take only properties with stereotypes or involved in reference relationships

In [ ]:
import json
finder = ['MovieDocument', 'ContributorDocument', 'RoleDocument']
for p in file_prod_impl_map['MovieDoc.xml']['products']:
    if p['name'] in finder:
        j = json.dumps(p, indent=2)
        print("\n\n**** Looking at ***** " + p['name'])
        print(j)


## Create RDF

In [ ]:
!pip install rdflib

In [28]:
import os
import json
from rdflib import Graph, Literal, RDF, RDFS, URIRef, XSD, OWL, BNode

NS = "http://amazon.com/aws/wwso/neptune/demo/multimodel"
DPURI = URIRef(f"{NS}/DataProduct")
IMPLURI = URIRef(f"{NS}/DataProductImpl")
MM_ONTOLOGY = URIRef(f"{NS}/ontology")
NIL = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#nil")


PKG = {
    "DataLake.xml": "lake", 
    "KnowledgeGraph.xml": "kg",
    "MovieDoc.xml": "moviedoc",
    "StoryAnalysis.xml": "story",
    "VideoAnalysis.xml": "video"
}

LITERAL_STEREOS = {
    "federates": {"tag": "fedURI", "obj": lambda val, umlfile, clazz, stereo: URIRef(val) } , 
    "awsService": {"tag": "service", "obj":  lambda val, umlfile, clazz, stereo: make_uri("aws", val) } , 
    "awsResource": {"tag": "resource", "obj":  lambda val, umlfile, clazz, stereo: Literal(val) } , #keep is loose for now
    "usagePattern": {"tag": "pattern", "obj":  lambda val, umlfile, clazz, stereo: Literal(val) } 
}

USAGE_STEREOS = ["copies", "caches", "locatedIn", "hasImpl"]

OREL_STEREOS = {
    "hasSource": {
        "orel": "Source",
        "tags": {
            "integrationType": {
                "p": lambda val, umlfile, clazz, stereo, target_file:make_top_uri("integrationType"), 
                "o": lambda val, umlfile, clazz, stereo, target_file: URIRef(make_top_uri(val))
            },
            "sourceDesc": {
                "p": lambda val, umlfile, clazz, stereo, target_file:RDFS.comment, 
                "o": lambda val, umlfile, clazz, stereo, target_file: Literal(val)
            },
            "sourceDataSet": {
                "p": lambda val, umlfile, clazz, stereo, target_file:make_top_uri("sourceDataSet"),
                "o": lambda val, umlfile, clazz, stereo, target_file: Literal(val)
            },
            "sourceEventType": {
                "p": lambda val, umlfile, clazz, stereo, target_file:make_top_uri("sourceEventType"),
                "o": lambda val, umlfile, clazz, stereo, target_file: make_uri("aws", val)
            }
        }
    } , 
    "similarTo": {
        "orel": "Similarity",
        "tags": {
            "simReason": {
                "p": lambda val, umlfile, clazz, stereo, target_file:make_top_uri("simReason"),
                "o": lambda val, umlfile, clazz, stereo, target_file: Literal(val)
            },
            "simAlgorithm": {
                "p": lambda val, umlfile, clazz, stereo, target_file:make_top_uri("simAlgorithm"),
                "o": lambda val, umlfile, clazz, stereo, target_file: Literal(val)   
            }
        }
    },
    "joins": {
        "orel": "Ref",
        "tags": {
            "joinAttrib": {
                "p": lambda val, umlfile, clazz, stereo, target_file: make_top_uri("hasNeighborAttribute"),
                "o": lambda val, umlfile, clazz, stereo, target_file: make_uri(PKG[target_file], val)
            }, 
            "myAttrib": {
                "p": lambda val, umlfile, clazz, stereo, target_file: make_top_uri("hasMyAttribute"),
                "o": lambda val, umlfile, clazz, stereo, target_file: make_uri(PKG[target_file], val)
            }
        }        
    },
    "refersTo": {
        "orel": "Ref",
        "tags": {
            "refersAttrib": {
                "p": lambda val, umlfile, clazz, stereo, target_file :make_top_uri("hasNeighborAttribute"),
                "o": lambda val, umlfile, clazz, stereo, target_file: make_uri(PKG[target_file], val)
            }, 
            "myAttrib": {
                "p": lambda val, umlfile, clazz, stereo, target_file: make_top_uri("hasMyAttribute"),
                "o": lambda val, umlfile, clazz, stereo, target_file: make_uri(PKG[umlfile], val)  
            }
        }        
    }
}

def make_top_uri(name):
    return URIRef(f"{NS}/{name}")

def make_uri(ns, name):
    return URIRef(f"{NS}/{ns}/{name}")

def create_prop_from_rel(g, puri, name):
    g.add((puri, RDF.type, OWL.DatatypeProperty))
    g.add((puri, RDFS.label, Literal(name)))
    g.add((puri, RDFS.isDefinedBy, MM_ONTOLOGY))
    return puri

def create_data_prop(g, ns, name, domain_clazz):
    puri = make_uri(ns, name)
    g.add((puri, RDF.type, OWL.DatatypeProperty))
    g.add((puri, RDFS.label, Literal(name)))
    g.add((puri, RDFS.isDefinedBy, MM_ONTOLOGY))
    g.add((puri, make_top_uri("domainIncludes"), domain_clazz))
    return puri

def create_data_type_prop(g, ns, name, propuri):
    puri = make_uri(ns, name)
    g.add((puri, RDF.type, OWL.DatatypeProperty))
    g.add((puri, RDFS.label, Literal(name)))
    g.add((puri, RDFS.isDefinedBy, MM_ONTOLOGY))
    g.add((propuri, RDFS.subPropertyOf, puri))
    return puri


def create_data_product(g, ns, name):
    puri = make_uri(ns, name)
    g.add((puri, RDF.type, OWL.Class))
    g.add((puri, RDFS.subClassOf, DPURI))
    g.add((puri, RDFS.label, Literal(name)))
    g.add((puri, RDFS.isDefinedBy, MM_ONTOLOGY))
    return puri

def create_impl(g, ns, name):
    puri = make_uri(ns, name)
    g.add((puri, RDF.type, OWL.Class))
    g.add((puri, RDFS.subClassOf, IMPLURI))
    g.add((puri, RDFS.label, Literal(name)))
    g.add((puri, RDFS.isDefinedBy, MM_ONTOLOGY))
    return puri
    
def add_config(g, cfg):
    g.add((cfg, RDFS.subClassOf, make_top_uri("config")))
    
def add_po(g, s, p, o):
    g.add((s, p, o))

def create_orel(g, oreltype, s, p, po):
    po_uri = BNode()
    g.add((s, p, po_uri))
    g.add((po_uri, RDF.type, OWL.NamedIndividual))
    g.add((po_uri, RDF.type, make_top_uri(oreltype)))
    g.add((po_uri, RDFS.isDefinedBy, MM_ONTOLOGY))
    for one_po in po:
        g.add((po_uri, one_po['p'], one_po['o']))

def convert_class(umlfile, clazz, s):
    # class-level stereotypes
    for stereo in clazz['stereotypes']:
        # the predicate
        sname = stereo['name']
        p = make_top_uri(sname)
        if sname in LITERAL_STEREOS:
            # TODO - some of the objects should be URIs, not literals
            tag = LITERAL_STEREOS[sname]['tag']
            obj = LITERAL_STEREOS[sname]['obj']
            if tag in stereo['tags']:
                for val in stereo['tags'][tag]:
                    add_po(g, s, p, obj(val, umlfile, clazz, stereo))
            else:
                print("Warn: incorrect tag structure in " + str(stereo))
                
        elif sname in USAGE_STEREOS:
            if len(stereo['targetClass']) > 0:
                o = make_uri(PKG[stereo['targetFile']], stereo['targetClass'])
                add_po(g, s, p, o)
            elif len(stereo['targetClasses']) > 0:
                for cl in stereo['targetClasses']:
                    o = make_uri(PKG[stereo['targetFile']], cl)
                    add_po(g, s, p, o)

        elif sname in OREL_STEREOS:
            po = []
            target_file = stereo['targetFile'] if 'targetFile' in stereo else None
            for t in OREL_STEREOS[sname]['tags']:
                if t in stereo['tags']:
                    for val in stereo['tags'][t]:
                        po_p = OREL_STEREOS[sname]['tags'][t]["p"](val, umlfile, clazz, stereo, target_file)
                        po_o = OREL_STEREOS[sname]['tags'][t]["o"](val, umlfile, clazz, stereo, target_file)
                        po.append({"p": po_p, "o": po_o})
                        if po_p == make_top_uri("hasNeighborAttribute") or po_o == make_top_uri("hasMyAttribute"):
                            create_prop_from_rel(g, po_o, val)
            if 'targetClass' in stereo and not (stereo['targetClass'] is None):
                po.append({"p": make_top_uri("hasNeighbor"), "o": make_uri(PKG[stereo['targetFile']], stereo['targetClass'])})
            elif 'targetClasses' in stereo and len(stereo['targetClasses']) > 0:
                for t in stereo['targetClasses']:
                    po.append({"p": make_top_uri("hasNeighbor"), "o": make_uri(PKG[stereo['targetFile']], t)})
            create_orel(g, OREL_STEREOS[sname]['orel'], s, p, po)

        elif sname == 'config':
            if 'configKV' in stereo['tags']:
                for kv in stereo['tags']['configKV']:
                    toks = kv.split(",")
                    if len(toks) == 2:
                        cp = make_uri("aws", "config-" + toks[0])
                        co = Literal(toks[1])
                        add_po(g, s, cp, co)
                        add_config(g, cp)
                    else:
                        print("Warn: illegal config in " + str(stereo))
        else:
            print("Warn: unknown stereotype " + str(stereo))
            
            
    # Now onto properties
    # 
    # We won't take all the properties. That's great input for data mesh, but we'll consider only a few properties
    # Products keys
    # Props with config (*** Not today, maybe later ***)
    # Properties used in join/refers
    #
    # Will not consider
    # Enums
    # Properties not dealt with above
    # Properties further down the tree.
    # 
    # How to handle types
    # Model as subClassOf instead of range
    # The IMDBID type is a great illustration
    # It is better to say "MovieID is subclass of IMDBID" than 
    # "MovieID has range IMDBID".
    keys = []
    for prop in clazz['properties']:
        prop_name = prop['name']
        for prop_stereo in prop['stereotypes']:
            propuri = create_data_prop(g, PKG[umlfile], prop_name, s)
            if prop_stereo['name'] == 'productKey':
                keys.append(propuri)
            if len(prop['type']) > 0:
                create_data_type_prop(g, PKG[umlfile], prop['type'], propuri)
                
    if len(keys) > 0:
        list_uri = BNode()
        add_po(g, s, OWL.hasKey, list_uri)
        idx = 0
        for k in keys:
            add_po(g, list_uri, RDF.first, k)
            idx += 1
            if idx == len(keys):
                add_po(g, list_uri, RDF.rest, NIL)
            else:
                next_list_uri = BNode()
                add_po(g, list_uri, RDF.rest, next_list_uri)  
                list_uri = next_list_uri

g = Graph()
for umlfile in file_prod_impl_map:
    for product in file_prod_impl_map[umlfile]['products']:
        print(product['name'])
        s =  create_data_product(g, PKG[umlfile], product['name'])
        convert_class(umlfile, product, s)
    for impl in file_prod_impl_map[umlfile]['impls']:
        print(impl['name'])
        s =  create_impl(g, PKG[umlfile], impl['name'])
        convert_class(umlfile, impl, s)
    
g.serialize(destination = 'mmgen/mmgen.ttl', format='turtle')

MovieLakeTable
ContributorLakeTable
RoleLakeTable
LakeStore
RoleTable
ContribTable
MovieTable
VideoAnalysis
VideoAnalysisBucket
AnalysisItem
IngestItem
VideoIngestBucket
StoryAnalysis
LAOrangeStoryBucket
RoleDocument
MovieDocument
ContributorDocument
MovieDocImpl
ContributorSearchDocument
MovieElasticacheRedisCache
RoleSearchDocument
ContribDocImpl
MovieSearchDocument
RoleDocImpl
VideoAnalysisResource
CelebResource
StoryResource
MovieResource
ContributorResource
NeptuneImpl


## Copy generated RDF to S3
We will be loading this into Neptune using the bulk loader. Let's stage it in S3.

In [29]:
%%bash -s "$STAGING_BUCKET"

cd mmgen
aws s3 cp mmgen.ttl s3://$1/data/mmgen.ttl

upload: ./mmgen.ttl to s3://nepmm4-s3bucket-9745jq3lxvw6/data/mmgen.ttl


## Upload three ontologies
1. From S3 data/mm.ttl - Data product ontology
2. From S3 data/mmgen.ttl - Movie products that we just generated from UML
3. From S3 data/movkg.ttl - Ontology of the movie KG data product

Run each and check the load status when it completes. Wait for the spinner to stop and status of LOAD_COMPLETED or LOAD_FAILED.

In [40]:
%load -s s3://{STAGING_BUCKET}/data/mm.ttl -f turtle --store-to loadres1 --run

Button(description='Submit', style=ButtonStyle())

Output()

In [33]:
%load_status {loadres1['payload']['loadId']} --errors --details

{
  "status": "200 OK",
  "payload": {
    "feedCount": [
      {
        "LOAD_COMPLETED": 1
      }
    ],
    "overallStatus": {
      "fullUri": "s3://nepmm4-s3bucket-9745jq3lxvw6/data/mm.ttl",
      "runNumber": 1,
      "retryNumber": 0,
      "status": "LOAD_COMPLETED",
      "totalTimeSpent": 3,
      "startTime": 1683838942,
      "totalRecords": 75,
      "totalDuplicates": 0,
      "parsingErrors": 0,
      "datatypeMismatchErrors": 0,
      "insertErrors": 0
    },
    "errors": {
      "startIndex": 0,
      "endIndex": 0,
      "loadId": "ad255716-bd5a-47a3-8d5e-1a12e73da5d8",
      "errorLogs": []
    }
  }
}


In [41]:
%load -s s3://{STAGING_BUCKET}/data/mmgen.ttl -f turtle --store-to loadres2 --run

Button(description='Submit', style=ButtonStyle())

Output()

In [35]:
%load_status {loadres2['payload']['loadId']} --errors --details

{
  "status": "200 OK",
  "payload": {
    "feedCount": [
      {
        "LOAD_COMPLETED": 1
      }
    ],
    "overallStatus": {
      "fullUri": "s3://nepmm4-s3bucket-9745jq3lxvw6/data/mmgen.ttl",
      "runNumber": 1,
      "retryNumber": 0,
      "status": "LOAD_COMPLETED",
      "totalTimeSpent": 3,
      "startTime": 1683838955,
      "totalRecords": 555,
      "totalDuplicates": 0,
      "parsingErrors": 0,
      "datatypeMismatchErrors": 0,
      "insertErrors": 0
    },
    "errors": {
      "startIndex": 0,
      "endIndex": 0,
      "loadId": "387621e1-2854-4cd6-908a-0401e72ef8b4",
      "errorLogs": []
    }
  }
}


In [42]:
%load -s s3://{STAGING_BUCKET}/data/movkg.ttl -f turtle --store-to loadres3 --run

Button(description='Submit', style=ButtonStyle())

Output()

In [37]:
%load_status {loadres3['payload']['loadId']} --errors --details

{
  "status": "200 OK",
  "payload": {
    "feedCount": [
      {
        "LOAD_COMPLETED": 1
      }
    ],
    "overallStatus": {
      "fullUri": "s3://nepmm4-s3bucket-9745jq3lxvw6/data/movkg.ttl",
      "runNumber": 1,
      "retryNumber": 0,
      "status": "LOAD_COMPLETED",
      "totalTimeSpent": 3,
      "startTime": 1683838970,
      "totalRecords": 537,
      "totalDuplicates": 10,
      "parsingErrors": 0,
      "datatypeMismatchErrors": 0,
      "insertErrors": 0
    },
    "errors": {
      "startIndex": 0,
      "endIndex": 0,
      "loadId": "2cc77bd7-12f6-4b91-b412-7ed8f73d4c78",
      "errorLogs": []
    }
  }
}


## Query the products (and some KG instances)

### Get list of products

In [45]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 

select ?product where {
    ?product rdfs:subClassOf :DataProduct .
} 
ORDER BY ?product 

### Get list of products and their impls

In [48]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 

select ?product (GROUP_CONCAT(?impl;SEPARATOR=",") AS ?impls) where {
    ?product rdfs:subClassOf :DataProduct .
    OPTIONAL { ?product :hasImpl+ ?impl } .
} 
GROUP BY ?product
ORDER BY ?product 

### Inspect one product a few levels deep

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

select * where {
    ?anno a owl:AnnotationProperty .
    movdoc:MovieDocument ?anno ?o .
    OPTIONAL { 
        ?anno2 a owl:AnnotationProperty  .
        ?o ?anno2 ?o2 . 
    } .
} 
ORDER BY ?anno ?o ?anno2 

### Describe a product

In [ ]:
%%sparql

# describe mode https://docs.aws.amazon.com/neptune/latest/userguide/sparql-query-hints-for-describe.html#sparql-query-hints-describeMode

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 
PREFIX hint: <http://aws.amazon.com/neptune/vocab/v01/QueryHints#>

describe movkg:MovieResource
#{
#  hint:Query hint:describeMode "CBD"
#}

### Which products use OpenSearch and Elasticache?

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

select * where {
    ?product rdfs:subClassOf :DataProduct .
    ?product :hasImpl/:hasService aws:OpenSearch .
    ?product :hasImpl/:hasService aws:Elasticache .
}

### Story and Movie Related?

In [ ]:
%%sparql
PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

ask where {
    BIND(movstory:StoryAnalysis as ?product) .
    
    ?product  ((:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf) |^ 
        (:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf))* ?rel .

    FILTER(?rel = movkg:MovieResource || ?rel = movdoc:MovieDocument) . 
} 


### Story and IMDB?

In [ ]:
%%sparql
PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

ask where {
    BIND(movstory:StoryAnalysis as ?product) .
    
    ?product  ((:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf) |^ 
        (:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf))* ?rel .

    FILTER(?rel = mov:IMDB) . 
} 

### Story and the Lonely product

In [ ]:
%%sparql
PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

ask where {
    BIND(movstory:StoryAnalysis as ?product) .
    
    ?product  ((:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf) |^ 
        (:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf))* ?rel .

    FILTER(?rel = mov:LonelyProduct) . 
} 

## Movie Example

### Populate sample data. A movie, a couple of its roles, stories that mention, video analysis, links to IMDB, DBPedia, Wikidata

In [ ]:
%%sparql
PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

INSERT DATA { graph <:mmblog> {
    movkg:Shining a movkg:MovieResource .
    movkg:Shining movdoc:MovieID "tt0081505" .
    movkg:Shining movkg:hasDBPediaRef <http://dbpedia.org/resource/The_Shining_(film)> .
    movkg:Shining movkg:hasWikidataRef <http://www.wikidata.org/entity/Q186341> .
    
    # cast - a couple contributors to give the idea
    movkg:RoleShining_Jack a movkg:RoleResource .
    movkg:RoleShining_Jack movkg:hasMovie movkg:Shining .
    movkg:RoleShining_Jack movkg:hasContribClass movkg:Actor .
    movkg:RoleShining_Jack movkg:hasContrib movkg:JackNicholson .

    movkg::RoleShining_Kubrick_Dir a movkg:RoleResource .
    movkg::RoleShining_Kubrick_Dir movkg:hasMovie movkg:Shining .
    movkg::RoleShining_Kubrick_Dir movkg:hasContribClass movkg:Director .
    movkg::RoleShining_Kubrick_Dir movkg:hasContrib movkg:StanleyKubrick .

    movkg:RoleShining_Kubrick_Prod a movkg:RoleResource .
    movkg:RoleShining_Kubrick_Prod movkg:hasMovie movkg:Shining .
    movkg:RoleShining_Kubrick_Prod movkg:hasContribClass movkg:Producer .
    movkg:RoleShining_Kubrick_Prob movkg:hasContrib movkg:StanleyKubrick .

    movkg:JackNicholson a movkg:ContributorResource . 
    movkg:JackNihcolson movkg:ContribID "nm0000197" .
    movkg:JackNicholson movkg:hasDBPediaRef <http://dbpedia.org/resource/Jack_Nicholson> .
    movkg:JackNicholson movkg:hasWikidataRef <https://www.wikidata.org/entitiy/Q39792> .

    movkg:StanleyKubrick a movkg:ContributorResource . 
    movkg:StanleyKubrick movkg:ContribID "nm0000040" .
    movkg:StanleyKubrick movkg:hasDBPediaRef <http://dbpedia.org/resource/Stanley_Kubrick> .
    movkg:StanleyKubrick movkg:hasWikidataRef <https://www.wikidata.org/entitiy/Q2001> .

    # stories that mention
    movkg:Story_Staycation_in_Hollywood a movkg:StorytResource .
    movkg:Story_Staycation_in_Hollywood movstory:StoryTitle "Staycation in Hollywood" .
    movkg:Story_Staycation_in_Hollywood movkg:mentions movkg:Shining .
    movkg:Story_Starve_Cabin_Fever_Until_Spring a movkg:StorytResource .
    movkg:Story_Starve_Cabin_Fever_Until_Spring movstory:StoryTitle "Starve Cabin Fever Until Spring" .
    movkg:Story_Starve_Cabin_Fever_Until_Spring movkg:mentions movkg:Shining .
    
    # video analysis
    movkg:Analysis_123456789 a movkg:VideoAnalysisResource .
    movkg:Shining movkg:hasVideoAnalysis movkg:Analysis_123456789 .
    movkg:Analysis_123456789 movvideo:VideoID "123456789" .
    movkg:Analysis_123456789 movvideo:S3IngestLocation "s3://va_abcderfg_123456789/ingest" .
    movkg:Analysis_123456789 movvideo:S3AnalysisLocation "s3://va_abcderfg_123456789/analysis" .
    movkg:Analysis_123456789 movvideo:MP4FileName "0081505_shining.mp4" .
    movkg:Analysis_123456789 movkg:hasRekognitionCeleb movkg:Analysis_123456789_celeb0 .
    movkg:Analysis_123456789_celeb0 movkg:celebName "Jeff Bezos" .
    movkg:Analysis_123456789_celeb0 movkg:hasWikidataRef <http://www.wikidata.org/entity/Q312556> .
    movkg:Analysis_123456789_celeb0 movdoc:ContribID "nm1757263" . # this is an IMDB ID
    # more detail on occurences of cebel in video in S3AnalysisLocation given above
 
}}

### With MovieID (IMDBID) as input, get basic details of the movie

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

SELECT ?movie ?dbp ?wiki ?storyMention ?video ?mp4
WHERE 
{
    ?movie movdoc:MovieID "tt0081505" .
    ?movie a movkg:MovieResource .
    OPTIONAL {?movie movkg:hasDBPediaRef ?dbp . } .
    OPTIONAL {?movie movkg:hasWikidataRef ?wiki . } .
    
    # bring in story mentions
    OPTIONAL {?storyMention movkg:mentions ?movie . } .
    
    # bring in video analyis
    OPTIONAL {?movie movkg:hasVideoAnalysis ?video . ?video movvideo:MP4FileName ?mp4 . } .
} 


### Knowing the movie URI, DESCRIBE it

See https://docs.aws.amazon.com/neptune/latest/userguide/sparql-query-hints-for-describe.html for more on DESCRIBE in Neptune. Try the Graph view too.

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

describe movkg:Shining
#{
#  hint:Query hint:describeMode "CBD"
#}

### Get video analysis - celebs

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

SELECT ?movie ?mp4 ?celebName ?celebWikdata ?celebIMDB ?roleX
WHERE 
{
    ?movie movdoc:MovieID "tt0081505" .
    ?movie a movkg:MovieResource .
    
    ?movie movkg:hasVideoAnalysis ?video . 
    ?video movvideo:MP4FileName ?mp4 .
    OPTIONAL {
        # bring in celebs in video analysis
        ?video movkg:hasRekognitionCeleb ?celeb .
        ?celeb movkg:celebName ?celebName .
        ?celeb movkg:hasWikidataRef ?celebWikdata .
        ?celeb movdoc:ContribID ?celebIMDB .
        OPTIONAL {
            # Is the celeb a contributor in the movie
            ?roleX movkg:hasContributor ?contribX .
            ?contribX a movkg:ContributorResource .
            ?roleX movkg:hasMovie ?movie .
            ?contribX movdoc:ContribID ?celebIMDB . 
        }
    } 
} 


### Pull in DBPedia

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

SELECT ?p ?o 
WHERE 
{
    ?movie movdoc:MovieID "tt0081505" .
    ?movie a movkg:MovieResource .
    ?movie movkg:hasDBPediaRef ?dbp .
    SERVICE <https://dbpedia.org/sparql> {
        ?dbp ?p ?o . 
    }
}


### Pull in Wikidata

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

SELECT ?p ?o 
WHERE 
{
    ?movie movdoc:MovieID "tt0081505" .
    ?movie a movkg:MovieResource .
    ?movie movkg:hasWikidataRef ?wiki .
    SERVICE <https://query.wikidata.org/sparql> {
        ?wiki ?p ?o . 
    }    
}


## Cleanup
If you messed up... 
Either of the two approaches works.

In [39]:
%%sparql

delete {?s ?p ?o} where {?s ?p ?o}

In [31]:
%db_reset

Checkbox(value=False, description='I acknowledge that upon deletion the cluster data will no longer be availab…

Output()